<a href="https://colab.research.google.com/github/TrucThanh777/TTNT_Thanhtruc/blob/main/Tuan04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bài 1: Phát triển code bài tập mẫu để đọc file ma trận kề dạng txt bất kỳ và in kết quả tô
màu ra màn hình.

In [ ]:
from google.colab import files
uploaded = files.upload()

# Lấy tên file vừa upload
filename = list(uploaded.keys())[0]
print("Đang dùng file:", filename)
##################################


Saving graph.txt to graph (1).txt
Đang dùng file: graph (1).txt


Hàm đọc ma trận và tô màu

In [ ]:
# -*- coding: utf-8 -*-

def read_adj_matrix(filename):
    """Đọc ma trận kề từ file txt.
       Mỗi dòng: các số cách nhau bởi khoảng trắng hoặc dấu phẩy.
    """
    G = []
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # bỏ dòng trống
            parts = line.replace(",", " ").split()
            row = [int(x) for x in parts]
            G.append(row)

    # kiểm tra ma trận vuông
    n = len(G)
    for r in G:
        if len(r) != n:
            raise ValueError("Ma trận kề không vuông (số dòng khác số cột).")

    return G


def graph_coloring_from_file(filename):
    # 1. Đọc ma trận kề
    G = read_adj_matrix(filename)
    n = len(G)

    # 2. Tên đỉnh: nếu n <= 26 dùng A, B, C, ...
    if n <= 26:
        node = [chr(ord('A') + i) for i in range(n)]
    else:
        node = [f"v{i+1}" for i in range(n)]

    # map tên đỉnh -> index
    t_ = {node[i]: i for i in range(n)}

    # 3. Bậc các đỉnh
    degree = [sum(G[i]) for i in range(n)]

    # 4. Các màu có thể dùng
    base_colors = ["Blue", "Red", "Yellow", "Green", "Orange", "Purple"]

    colorDict = {node[i]: base_colors.copy() for i in range(n)}

    # 5. Sắp xếp đỉnh theo bậc giảm dần (selection sort)
    sortedNode = []
    used_idx = []

    for _ in range(len(degree)):
        _max = -1
        idx = -1
        for j in range(len(degree)):
            if j not in used_idx and degree[j] > _max:
                _max = degree[j]
                idx = j
        used_idx.append(idx)
        sortedNode.append(node[idx])

    # 6. Tô màu tham lam
    theSolution = {}

    for v in sortedNode:
        available_colors = colorDict[v]
        if not available_colors:
            # nếu hết màu thì tạo màu mới (phòng TH cần > số màu gốc)
            new_color = f"Color_{len(base_colors)+1}"
            base_colors.append(new_color)
            available_colors.append(new_color)

        chosen_color = available_colors[0]
        theSolution[v] = chosen_color

        # danh sách kề của v
        adj_row = G[t_[v]]

        # xóa màu này khỏi các đỉnh kề
        for j, value in enumerate(adj_row):
            if value == 1:
                u = node[j]
                if chosen_color in colorDict[u]:
                    colorDict[u].remove(chosen_color)

    # 7. In kết quả
    print("=== KẾT QUẢ TÔ MÀU ĐỒ THỊ ===")
    print(f"Số đỉnh: {n}")
    print("Đỉnh : Màu")
    for v in sorted(theSolution.keys()):
        print(f"{v} : {theSolution[v]}")


chạy chương trình

In [ ]:
graph_coloring_from_file(filename)


=== KẾT QUẢ TÔ MÀU ĐỒ THỊ ===
Số đỉnh: 6
Đỉnh : Màu
A : Yellow
B : Blue
C : Red
D : Yellow
E : Blue
F : Red


Bài 2: phát triển đọc ma trận kề, tạo tên đỉnh, tô màu tham lam, in kết quả

In [ ]:
# ====== 1. Đọc ma trận kề từ file ======
def read_adj_matrix(filename):
    """
    Đọc ma trận kề từ file .txt
    Mỗi dòng: các số cách nhau bởi khoảng trắng hoặc dấu phẩy.
    """
    G = []
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.replace(",", " ").split()
            row = [int(x) for x in parts]
            G.append(row)

    # kiểm tra ma trận vuông
    n = len(G)
    for r in G:
        if len(r) != n:
            raise ValueError("Ma trận kề không vuông.")
    return G


# ====== 2. Tạo tên các đỉnh ======
def create_node_names(n):
    """
    Tạo tên đỉnh:
    - Nếu n <= 26: A, B, C, ...
    - Nếu n > 26: v1, v2, ...
    """
    if n <= 26:
        return [chr(ord('A') + i) for i in range(n)]
    else:
        return [f"v{i+1}" for i in range(n)]


# ====== 3. Tô màu đồ thị bằng thuật toán tham lam ======
def greedy_coloring(G, node, color_list):
    """
    Tô màu đồ thị:
    - G: ma trận kề
    - node: danh sách tên đỉnh
    - color_list: danh sách màu ban đầu
    Trả về dict: {đỉnh: màu}
    """
    n = len(node)

    # map tên đỉnh -> index trong ma trận
    index_of = {node[i]: i for i in range(n)}

    # bậc + sắp xếp đỉnh
    degree = compute_degrees(G)
    sorted_nodes = sort_nodes_by_degree(node, degree)

    # mỗi đỉnh có list màu khả dụng
    colorDict = {v: color_list.copy() for v in node}

    solution = {}

    for v in sorted_nodes:
        available = colorDict[v]

        # nếu hết màu còn dùng được thì tạo màu mới
        if not available:
            new_color = f"Color_{len(color_list)+1}"
            color_list.append(new_color)
            available.append(new_color)

        chosen_color = available[0]
        solution[v] = chosen_color

        # loại bỏ màu này khỏi các đỉnh kề
        row = G[index_of[v]]
        for j, value in enumerate(row):
            if value == 1:          # có cạnh (v, u)
                u = node[j]
                if chosen_color in colorDict[u]:
                    colorDict[u].remove(chosen_color)

    return solution


# ====== 4. In kết quả tô màu ======
def print_coloring(solution):
    """
    In kết quả tô màu ra màn hình.
    """
    print("=== KẾT QUẢ TÔ MÀU ĐỒ THỊ ===")
    for v in sorted(solution.keys()):
        print(f"{v} : {solution[v]}")


# ====== 5. Hàm main điều khiển ======
def main(filename):
    # 1. Đọc ma trận kề
    G = read_adj_matrix(filename)

    # 2. Tạo tên đỉnh
    n = len(G)
    node = create_node_names(n)

    # 3. Danh sách màu ban đầu
    base_colors = ["Blue", "Red", "Yellow", "Green", "Orange", "Purple"]

    # 4. Tô màu
    solution = greedy_coloring(G, node, base_colors)

    # 5. In kết quả
    print_coloring(solution)
    main(filename)


bài 3

In [ ]:
INF = 10**18   # "vô cực" lớn để so sánh


def read_cost_matrix():
    """
    Đọc dữ liệu từ bàn phím:
    - n: số thành phố
    - ma trận chi phí n x n
    Trả về (n, cost)
    """
    n = int(input("Nhập số thành phố n: "))
    print(f"Nhập ma trận chi phí {n} x {n} (c[i][j]):")
    cost = []
    for i in range(n):
        row = list(map(int, input().split()))
        if len(row) != n:
            raise ValueError("Số phần tử mỗi dòng phải đúng n.")
        cost.append(row)
    return n, cost


def tsp_branch_and_bound(n, cost):
    """
    Giải bài toán người bán hàng:
    - đi qua mỗi thành phố đúng 1 lần
    - quay về thành phố xuất phát (0)
    - tổng chi phí nhỏ nhất

    Trả về: (best_cost, best_path)
    best_path là list các chỉ số thành phố (0-based).
    """

    # tìm cạnh có chi phí nhỏ nhất (cmin) để dùng cho nhánh cận
    cmin = INF
    for i in range(n):
        for j in range(n):
            if i != j and cost[i][j] > 0 and cost[i][j] < cmin:
                cmin = cost[i][j]

    best_cost = INF
    best_path = []

    path = [0] * n        # path[k] = thành phố tại vị trí k (0-based)
    visited = [False] * n

    path[0] = 0           # chọn thành phố 0 làm điểm xuất phát
    visited[0] = True
    current_cost = 0

    def Try(k):
        nonlocal current_cost, best_cost, best_path

        # đang chọn path[k]
        for v in range(1, n):       # các thành phố còn lại (1..n-1)
            if not visited[v]:
                path[k] = v
                visited[v] = True
                current_cost += cost[path[k - 1]][v]

                if k == n - 1:
                    # đã đi qua đủ n thành phố, cộng chi phí quay về 0
                    total = current_cost + cost[v][0]
                    if total < best_cost:
                        best_cost = total
                        best_path = path.copy()
                else:
                    # nhánh cận:  còn (n-k) cạnh nữa (k+1..n-1, rồi quay về 0)
                    lower_bound = current_cost + (n - k) * cmin
                    if lower_bound < best_cost:
                        Try(k + 1)

                # backtrack
                visited[v] = False
                current_cost -= cost[path[k - 1]][v]

    Try(1)
    return best_cost, best_path


def main():
    n, cost = read_cost_matrix()
    best_cost, best_path = tsp_branch_and_bound(n, cost)

    if best_cost >= INF:
        print("Không tìm được chu trình hợp lệ.")
    else:
        # đổi sang đánh số thành phố 1..n cho dễ nhìn
        route = [city + 1 for city in best_path] + [1]  # quay về 1
        print("\nChi phí nhỏ nhất:", best_cost)
        print("Chu trình tối ưu:", " -> ".join(map(str, route)))


if __name__ == "__main__":
    main()


Nhập số thành phố n: 4
Nhập ma trận chi phí 4 x 4 (c[i][j]):
0 2 9 10  
1 0 6 4
15 7 0 8
6 3 12 0

Chi phí nhỏ nhất: 21
Chu trình tối ưu: 1 -> 3 -> 4 -> 2 -> 1
